# LangGraph 体验

## Chapter 1: LangGraph 基本 Agent 构建

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

LLM_NAME = os.getenv("ZHIPU_LLM_NAME") or "glm-4-flash"
API_BASE = os.getenv("ZHIPU_API_BASE") or None
API_KEY = os.getenv("ZHIPU_API_KEY") or None

### 第一步：定义当前状态

“状态”，类似于 vuex 当中的 state，是整个 Agent 运行过程中的数据存储和传递的载体。

在 LangGraph 中，状态是一个继承了 TypedDict 的类，其中的每一个属性都是一个状态变量，一般情况下，里面得有一个 messages 变量，用于存储消息；这个 messages 变量必须是一个 Annotated 类型并打上 add_messages 标记。

直接就当做下面的是基本模板，是定义 graph 必备内容即可。后续可以根据业务需求，进一步添加其他的状态变量。

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]


### 第二步：定义 LLM 及 LLM 的绑定工具

此步骤略，同 LangChain。其中值得注意的是，绑定工具指的是每一次调用 LLM 都强制会被调用的、随后工具结果强硬的被添加在历史记录中的工具。下文以 DuckDuckGoSearchRun 工具为例。

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchRun

wrapper = DuckDuckGoSearchAPIWrapper(max_results=5)
tool = DuckDuckGoSearchRun(api_wrapper=wrapper)
tools = [tool]
llm = ChatOpenAI(model=LLM_NAME, openai_api_key=API_KEY, openai_api_base=API_BASE)
llm_with_tools = llm.bind_tools(tools)


### 第三步：定义图和 LLM 节点

在 LangGraph 中，Agent 的构建及消息数据处理决策流被抽象成一个有向图，其中节点是一个个处理过程（以状态为单一参数的普通 Python 函数），边是每一个处理过程的数据流向。

要构建 Agent，首先需要定义一个 Graph 对象。

在 LangGraph 中，定义的 Graph 对象是一个 StateGraph 类对象。构造 StateGraph 类对象时，必须先提供一个状态类（本例中的状态类为 State 类）。好在我们前文中已经定义过了这个状态类。

状态将是最终构建的“有向图”中的信息载体，随数据流而传递变化，为每一个节点提供数据、决策依据。

In [ ]:
from langgraph.graph import StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, tools_condition

graph_builder = StateGraph(State)  # 构建 StateGraph 对象
memory = MemorySaver()  # 构建消息记录存储器，可被换成 Redis 或 Postgres


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)  # 添加 chatbot 节点

tool_node = ToolNode(tools=[tool])  # 这里以再次提供 DuckDuckGoSearchRun 工具为例，实际业务中，前文已经强迫调用过了，这里应该是其他工具
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(  # 条件边，chatbot -> tools 或 chatbot -> __end__
    "chatbot",
    tools_condition,  # prebuilt.tools_condition 的决策依据是 LLM 自己选，如果不是 tools 就是 end
)
graph_builder.add_edge("tools", "chatbot")  # tools -> chatbot
graph_builder.set_entry_point("chatbot")  # 起始点是 chatbot
graph = graph_builder.compile(checkpointer=memory)  # 编译成有向图

#### 第四步：运行 Agent

通过前面的步骤，我们已经通过构建有向图的方式，构建了我们自己的 Agent，并配置好了一个基于内存存储的消息记录器。

下面我们讲通过 stream() 方法，运行我们的带有聊天记录记忆的 Agent。


In [ ]:
config = {"configurable": {"thread_id": "1"}}  # 配置项，相当于 LangChain 当中用于消息记录存取的 session_id
user_input = ("你可以告诉我什么是langgraph吗？")

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"  # 其中的参数待研究
)
for event in events:
    event["messages"][-1].pretty_print()  # 该方法待研究

至此，我们已经完成了 LangGraph 的基本 Agent 的构建流程，完成了我们的初次新 Agent 方法构建体验！

LangGraph 相比起传统的 LangChain Agent，最大的优势在于可以在 Agent 构建过程中，灵动的调整工具之间的调用关系。

LangGraph 中以 node、edge、condition、state 为关键元素构建有向图的方式，使得它可以有以下几种有趣的工作特点：

- 可中断工作流，可人为调整工作流
- 构建复杂而允许可视化的工作流
- 每一阶段数据都能被感知、存储
- 可以很方便的以“用户”作为重要决策环节

通过上面的示例，可以很好地看出、体验到 LangChain 在构建 Agent 的关键思想和相关工具调用方式，在下一章节中，我们将进一步跟随官方文档，体验以“用户”为决策环节的“工具”和更多有关 LangGraph 的高级特性！
